In [20]:
import pandas as pd

# Load the loan and BTC data
loan_file_path = r"C:\Users\ebuca\OneDrive\Documents\DA13\Python\projects\Capstone_project\updated_loan_data.csv"
btc_file_path = r"C:\Users\ebuca\OneDrive\Documents\DA13\Python\projects\Capstone_project\btc-usd-max.csv"

loan_data = pd.read_csv(loan_file_path)  # Loan data
btc_data = pd.read_csv(btc_file_path)  # BTC price data

# Convert dates to datetime format
loan_data['issue_date'] = pd.to_datetime(loan_data['issue_date'])
loan_data['payoff_date'] = pd.to_datetime(loan_data['payoff_date'])
btc_data['bitcoin_date'] = pd.to_datetime(btc_data['bitcoin_date'])

# Ensure BTC data is sorted for merge_asof and forward filling missing prices
btc_data = btc_data.sort_values('bitcoin_date')
btc_data['one_dollar_sats'] = btc_data['one_dollar_sats'].ffill()  # Corrected


# Drop missing issue or payoff dates
loan_data = loan_data.dropna(subset=['issue_date', 'payoff_date'])

# Initialize list to store results
customer_sats_per_month = []

# Iterate through each customer
for index, row in loan_data.iterrows():
    name = row['Names']
    issue_date = row['issue_date']
    payoff_date = row['payoff_date']
    monthly_investment = row['ten_of monthly']

    # Start from the next month
    start_date = issue_date + pd.DateOffset(months=1)
    months = pd.date_range(start=start_date, end=payoff_date, freq='MS')  # 'MS' = Month Start

    cumulative_satoshis = 0  # Track cumulative sats

    for month in months:
        # Get the Bitcoin data for the corresponding month
        btc_price_row = btc_data.loc[btc_data['bitcoin_date'] == month]

        if not btc_price_row.empty:
            one_dollar_sats = btc_price_row.iloc[0]['one_dollar_sats']
        else:
            one_dollar_sats = btc_data['one_dollar_sats'].median()  # Use median if no data
        
        # Calculate satoshis bought that month
        satoshis_bought = monthly_investment * one_dollar_sats
        cumulative_satoshis += satoshis_bought  # Add to cumulative total

        # Get the current Bitcoin price
        if not btc_price_row.empty:
            current_btc_price = btc_price_row.iloc[0]['price']
        else:
            current_btc_price = btc_data['price'].median()  # Use median BTC price if missing

        # Calculate current dollar value of accumulated satoshis
        current_dollar_value = (cumulative_satoshis / 100_000_000) * current_btc_price

        # Store result for this customer and month
        customer_sats_per_month.append({
            'Name': name,
            'Month': month.strftime('%Y-%m'),
            'Monthly Satoshis': satoshis_bought,
            'Cumulative Satoshis': cumulative_satoshis,
            'Bitcoin Price (USD)': current_btc_price,
            'Current Dollar Value': current_dollar_value
        })

# Convert list to DataFrame
sats_accumulation = pd.DataFrame(customer_sats_per_month)

# Display first few rows
print(sats_accumulation.head(40))


# ✅ Save the dataset as a CSV file
sats_accumulation.to_csv("sats_accumulation.csv", index=False)

# ✅ Print a confirmation message
print("CSV file saved successfully as 'sats_accumulation.csv'")




            Name    Month  Monthly Satoshis  Cumulative Satoshis  \
0   Sonya Hudson  2018-03          802195.0             802195.0   
1   Sonya Hudson  2018-04         1189888.0            1992083.0   
2   Sonya Hudson  2018-05          902376.0            2894459.0   
3   Sonya Hudson  2018-06         1112615.0            4007074.0   
4   Sonya Hudson  2018-07         1303681.0            5310755.0   
5   Sonya Hudson  2018-08         1072609.0            6383364.0   
6   Sonya Hudson  2018-09         1181090.0            7564454.0   
7   Sonya Hudson  2018-10         1252636.0            8817090.0   
8   Sonya Hudson  2018-11         1303681.0           10120771.0   
9   Sonya Hudson  2018-12         2070020.0           12190791.0   
10  Sonya Hudson  2019-01         2247806.0           14438597.0   
11  Sonya Hudson  2019-02         2418454.0           16857051.0   
12  Sonya Hudson  2019-03         2175264.0           19032315.0   
13  Sonya Hudson  2019-04         2022627.0     